In [ ]:
import numpy as np
import tensorflow as tf
from scipy.integrate import solve_ivp
import pickle
import pysindy as ps
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import ParameterGrid
import os
from torchdiffeq import odeint
import matplotlib.pyplot as plt
from torch.fft import fft, ifft
from mpl_toolkits.mplot3d import Axes3D
import optuna
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import MultiTaskLassoCV, LinearRegression
from sklearn.feature_selection import SelectFromModel
from typing import Tuple, Any, Iterable, Dict
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
def generate_lorenz_data(timesteps, dt, initial_conditions, sigma=10, beta=8/3, rho=28):
    """
    Generate Lorenz system data.

    :param timesteps: Number of timesteps to simulate.
    :param dt: Time step size.
    :param initial_conditions: Initial conditions (x0, y0, z0).
    :param sigma: Lorenz system parameter.
    :param beta: Lorenz system parameter.
    :param rho: Lorenz system parameter.
    :return: Numpy array of shape (timesteps, 3).
    """
    def lorenz(t, state):
        x, y, z = state
        dxdt = sigma * (y - x)
        dydt = x * (rho - z) - y
        dzdt = x * y - beta * z
        return [dxdt, dydt, dzdt]

    t_eval = np.linspace(0, dt * timesteps, timesteps)
    sol = solve_ivp(lorenz, [0, dt * timesteps], initial_conditions, t_eval=t_eval, method='RK45')
    return sol.y.T  # Transpose to shape (timesteps, 3)

# Generate dataset parameters
timesteps = 7500  # Extended sequence length for better visualization
n_samples = 20000  # Total number of sequences
dt = 0.01  # Time step size

# Generate data
np.random.seed(42)
initial_conditions_list = np.random.uniform(-10, 10, size=(n_samples, 3))  # Initial conditions closer to attractor

data = np.array([generate_lorenz_data(timesteps, dt, ic) for ic in initial_conditions_list])

In [ ]:
# Split data into train, validation, and test sets
train_data = data[:16000]
val_data = data[16000:18000]
test_data = data[18000:]

In [ ]:
print("\nExamples from the dataset:")
fig = plt.figure(figsize=(15, 10))
for i in range(10):
    example = data[i]
    print(example)
    ax = fig.add_subplot(2, 5, i + 1, projection='3d')
    ax.plot(example[:, 0], example[:, 1], example[:, 2])
    ax.set_title(f"Example {i + 1}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

plt.tight_layout()
plt.show()

In [ ]:
batch_size = 64

def prepare_dataloader(data, batch_size):
    inputs = torch.tensor(data[:, :-1, :], dtype=torch.float32)  # All but last timestep as input
    targets = torch.tensor(data[:, -1, :], dtype=torch.float32)  # Last timestep as target
    dataset = TensorDataset(inputs, targets)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

train_loader = prepare_dataloader(train_data, batch_size)
val_loader = prepare_dataloader(val_data, batch_size)
test_loader = prepare_dataloader(test_data, batch_size)

In [ ]:
class AdvancedConvFNO(nn.Module):
    def __init__(self, modes, width, conv_filters):
        super(AdvancedConvFNO, self).__init__()
        self.modes = modes
        self.width = width

        # Convolutional Layers
        self.conv1 = nn.Conv1d(3, conv_filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(conv_filters, conv_filters, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(conv_filters, self.width, kernel_size=3, padding=1)

        # Fourier Neural Operator
        self.fc0 = nn.Linear(self.width, self.width)
        self.fc1 = nn.Linear(self.width, self.width)
        self.fc2 = nn.Linear(self.width, 3)  # Output (x, y, z)

    def forward(self, x):
        batchsize, seq_len, channels = x.shape

        # Apply convolutions
        x = x.permute(0, 2, 1)  # Change to (batch, channels, seq_len)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.permute(0, 2, 1)  # Back to (batch, seq_len, channels)

        # Fourier Neural Operator
        x = self.fc0(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x[:, -1, :])  # Use the last timestep
        return x

In [ ]:
# Define SINDy for Post-Training Analysis
def apply_sindy(train_data_flat, test_data_flat, dt):
    x_train = train_data_flat[:-1]
    dx_train = (train_data_flat[1:] - train_data_flat[:-1]) / dt
    x_test = test_data_flat[:-1]
    dx_test = (test_data_flat[1:] - test_data_flat[:-1]) / dt

    # Define the SINDy model
    feature_library = ps.PolynomialLibrary(degree=3)
    optimizer = ps.STLSQ(threshold=0.1)
    sindy_model = ps.SINDy(feature_library=feature_library, optimizer=optimizer)

    # Fit and evaluate
    sindy_model.fit(x_train, t=dt, x_dot=dx_train)
    print("\nSINDy Discovered Equations:")
    sindy_model.print()
    print(f"SINDy Test Score: {sindy_model.score(x_test, t=dt, x_dot=dx_test):.4f}")

# Accuracy Calculation
def calculate_accuracy(outputs, targets, tolerance=0.1):
    correct = torch.sum(torch.abs(outputs - targets) < tolerance).item()
    total = targets.numel()
    return correct / total * 100

In [ ]:
# Training Loop with Accuracy
def train_and_evaluate(model, train_loader, val_loader, epochs, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_correct += torch.sum(torch.abs(outputs - targets) < 0.1).item()
            train_total += targets.numel()

        train_accuracy = train_correct / train_total * 100

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                val_correct += torch.sum(torch.abs(outputs - targets) < 0.1).item()
                val_total += targets.numel()

        val_accuracy = val_correct / val_total * 100

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Train Accuracy: {train_accuracy:.2f}%, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, "
              f"Val Accuracy: {val_accuracy:.2f}%")

    return val_loss / len(val_loader)

In [ ]:
# Hyperparameter Grid
param_grid = {
    'modes': [16, 32],
    'width': [64, 128],
    'conv_filters': [16, 32],
    'learning_rate': [0.001, 0.0005],
    'epochs': [50]
}

best_model = None
best_loss = float('inf')
best_params = None

# Grid Search for Hyperparameters
for params in ParameterGrid(param_grid):
    print(f"Testing params: {params}")
    model = AdvancedConvFNO(modes=params['modes'], width=params['width'], conv_filters=params['conv_filters'])
    val_loss = train_and_evaluate(model, train_loader, val_loader,
                                  epochs=params['epochs'],
                                  learning_rate=params['learning_rate'])
    if val_loss < best_loss:
        best_loss = val_loss
        best_model = model
        best_params = params

print(f"Best Params: {best_params}, Best Validation Loss: {best_loss:.4f}")

In [ ]:
def advanced_sindy(train_data_flat, test_data_flat, dt):
    x_train = train_data_flat[:-1]
    dx_train = (train_data_flat[1:] - train_data_flat[:-1]) / dt
    x_test = test_data_flat[:-1]
    dx_test = (test_data_flat[1:] - test_data_flat[:-1]) / dt

    # Define multiple libraries to test
    polynomial_degrees = [2, 3, 4, 5]  # Include higher polynomial degrees
    thresholds = np.linspace(0.01, 0.2, 10)  # Finer threshold range
    n_frequencies = [3, 5, 7]  # Test different numbers of Fourier frequencies

    best_model = None
    best_score = -float("inf")
    best_params = {}

    for degree in polynomial_degrees:
        for threshold in thresholds:
            for freq in n_frequencies:
                print(f"Testing SINDy with Polynomial Degree: {degree}, Threshold: {threshold}, Frequencies: {freq}")

                # Define the SINDy model
                feature_library = ps.PolynomialLibrary(degree=degree) + ps.FourierLibrary(n_frequencies=freq)
                optimizer = ps.STLSQ(threshold=threshold)
                sindy_model = ps.SINDy(
                    optimizer=optimizer,
                    feature_library=feature_library,
                    differentiation_method=ps.SmoothedFiniteDifference()  # Use smoother differentiation
                )

                # Fit the model
                sindy_model.fit(x_train, t=dt, x_dot=dx_train)
                score = sindy_model.score(x_test, t=dt, x_dot=dx_test)

                if score > best_score:
                    best_model = sindy_model
                    best_score = score
                    best_params = {"degree": degree, "threshold": threshold, "frequencies": freq}

    # Print the best model's equations
    print("\nBest SINDy Model:")
    best_model.print()
    print(f"Best SINDy Test Score: {best_score:.4f}")
    print(f"Best Parameters: {best_params}")

    # Evaluate on the test set
    test_score = best_model.score(x_test, t=dt, x_dot=dx_test)
    print(f"Final SINDy Test Score: {test_score:.4f}")

    return best_model, best_params, test_score

# Apply the updated SINDy
best_sindy_model, best_params, test_score = advanced_sindy(train_data_flat, test_data_flat, dt=0.01)

In [ ]:
def test_and_compare(best_model, best_sindy_model, test_loader, dt):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_model.to(device)

    best_model.eval()
    all_model_predictions = []
    all_ground_truth = []
    all_sindy_predictions = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.cpu().numpy()
            outputs = best_model(inputs).cpu().numpy()
            
            # Collect predictions and ground truth
            all_model_predictions.append(outputs)
            all_ground_truth.append(targets)
            
            # SINDy predictions
            test_inputs_flat = inputs.cpu().numpy().reshape(-1, inputs.shape[-1])
            sindy_predictions = best_sindy_model.predict(test_inputs_flat)
            all_sindy_predictions.append(sindy_predictions[-targets.shape[0]:])  # Use last part of the prediction

    # Convert lists to arrays
    all_model_predictions = np.concatenate(all_model_predictions, axis=0)
    all_ground_truth = np.concatenate(all_ground_truth, axis=0)
    all_sindy_predictions = np.concatenate(all_sindy_predictions, axis=0)

    # Calculate errors
    model_mae = mean_absolute_error(all_ground_truth, all_model_predictions)
    sindy_mae = mean_absolute_error(all_ground_truth, all_sindy_predictions)
    
    print(f"Model Mean Absolute Error: {model_mae:.4f}")
    print(f"SINDy Mean Absolute Error: {sindy_mae:.4f}")

    # Visualize comparisons for a few samples
    num_samples_to_plot = 5
    fig = plt.figure(figsize=(15, 10))
    for i in range(num_samples_to_plot):
        ax = fig.add_subplot(1, num_samples_to_plot, i + 1, projection='3d')

        # Ground truth
        ax.plot(
            all_ground_truth[i, :, 0],
            all_ground_truth[i, :, 1],
            all_ground_truth[i, :, 2],
            'g',
            label="Ground Truth"
        )
        # Model predictions
        ax.plot(
            all_model_predictions[i, :, 0],
            all_model_predictions[i, :, 1],
            all_model_predictions[i, :, 2],
            'b',
            label="Model Predictions"
        )
        # SINDy predictions
        ax.plot(
            all_sindy_predictions[i, :, 0],
            all_sindy_predictions[i, :, 1],
            all_sindy_predictions[i, :, 2],
            'r',
            label="SINDy Predictions"
        )
        ax.set_title(f"Sample {i + 1}")
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        ax.legend()

    plt.tight_layout()
    plt.show()


# Prepare the dataloader for the test dataset
test_loader = prepare_dataloader(test_data, batch_size)

# Compare the predictions of the best model and SINDy model
test_and_compare(best_model, best_sindy_model, test_loader, dt=0.01)